In [3]:
from common import *
from search.imports import *

## File System

In [ ]:
class IndexFileSystem():
    def __init__(self, directory, key):
        self.dir = directory if isinstance(directory, Path) else Path(directory)
        self.dir.mkdir(exist_ok=True, parents=True)
        assert isinstance(key, str), "key must be string"
        self.key = key
        self.io = O()

    def iterIndices(self):
        for dot_params in self.dir.glob('*.' + self.key):
            yield int(dot_params.stem)

    def getFilePath(self, name, *, i):
        return self.dir / (str(i) + '.' + name)

    @staticmethod
    def readWrapper(read):
        '''wraps io read operations to safely return None if file does not exist'''
        @wraps(read)
        def read_safely(*a, **k):
            try:
                return read(*a, **k)
            except FileNotFoundError:
                return None
        return read_safely

    def assignIO(self, name, *, read, write, format='custom'):
        assert format in ['bytes', 'text', 'custom'], "argument `format` must be one of 'bytes' or 'text' or 'custom'"
        if format in ['bytes', 'text']:
            self.io[name] = O(
                read = self.readWrapper( lambda *,i: read(getattr(self.getFilePath(name, i=i), 'read_'+format)()) ),
                write = lambda x,*,i: getattr(self.getFilePath(name, i=i), 'write_'+format)(write(x))
            )
        elif format == 'custom':
            def readCustom(*, i):
                file = self.getFilePath(name, i=i)
                if not file.exists():
                    return None
                try:
                    return read(file)
                except Exception:
                    return read(str(file))
            def writeCustom(*, i):
                file = self.getFilePath(name, i=i)
                try:
                    write(x, file=file)
                except Exception:
                    write(x, file=str(file))
            self.io[name] = O(read=readCustom, write=writeCustom)

class IndexDataStore():    
    def __init__(self, file, _factory_=False):
        if not _factory_:
            assert False, "Cannot instantiate IndexDataStore normally. Please use factory static method."
        self.file = file
        self.op = O()
        self.lists = O()
        self.key = None
        self.keyFunc = lambda keyVal, *, client=False: object()
        self.tbl = {}
        self.nextIndex = 0
        
    def load(self):
        indices = sorted(self.file.iterIndices())
        n = self.nextIndex = (max(indices) if indices else -1) + 1
        for name, _ in dict.items(self.lists):
            self.lists[name] = [None] * self.nextIndex
        
        for i in indices:
            for name in self.lists:
                if self.op[name].load:
                    self.op[name].load(i=i)
                
        if self.key is not None:
            self.assignKey(self.key, self.keyFunc)
            
    def edit(self, name, map):
        for i in sorted(self.file.iterIndices()):
            value = self.get(name, i=i)
            value = map(value)
            self.op[name].save(value, i=i)
            
    def i(self, key=None, *, dry=False, client=True, keep=True, save=True, **kw):
        if len(kw) > 0:
            assert len(kw)==1, 'only keyword arg allowed to method `i` is the key name'
            assert self.key in kw, 'only keyword arg allowed to method `i` is the key name'
            assert key is None, 'argument `key` already given as keyword using the key name'
            key = kw[self.key]
        else:
            assert key is not None, '`key` must be given by position if keyword with corresponding key name is not given'
        
        keyVal = self.keyFunc(key, client=client)
        
        if keyVal not in self.tbl and not dry:
            for name in self.lists:
                assert len(self.lists[name]) == self.nextIndex, "idk internal error"
                self.lists[name].append(None)
            self.tbl[keyVal] = self.nextIndex
            if save and self.op[self.key].save:
                self.op[self.key].save(key, i=self.nextIndex, keep=keep)
            elif keep and self.op[self.key].keep:
                self.op[self.key].keep(key, i=self.nextIndex, client=client)
            self.nextIndex += 1
        elif keyVal not in self.tbl and dry:
            return None
        
        return self.tbl[keyVal]
            
    def save(self, keep=True, **kwargs):
        '''`client` is always True'''
        assert ('i' in kwargs) ^ (self.key in kwargs), "call to `save` must include exactly one of i= or the key name ="
        if 'i' in kwargs:
            i = kwargs['i']
            del kwargs['i']
        else:
            i = self.i(kwargs[self.key])
        
        for name in kwargs:
            if name == self.key: # already saved in `self.i(save=True)` above
                continue
            if name not in self.op:
                raise AssertionError(f"given save item '{name}' does not have data store ops initialized")
            if self.op[name].save:
                self.op[name].save(kwargs[name], i=i, keep=keep)
        
        return i
            
    def get(self, name, **kwargs):
        assert len(kwargs) == 1 and ('i' in kwargs or self.key in kwargs), "must give assigned 'key' or i"
        if 'i' in kwargs:
            i = kwargs['i']
            del kwargs['i']
        else:
            i = self.i(kwargs[self.key], dry=True)
        if i is None:
            return None
        got = self.lists[name][i]
        if got is None and self.op[name].load:
            got = self.op[name].load(i=i)
        return got
        
    ##################### INSTANCE BUILDING METHODS ##################### : 
    def assignKey(self, name, func):
        assert name in self.lists, "key name must first be assigned ops"
        self.key = name
        self.keyFunc = func
        self.tbl = {self.keyFunc(x): i for i,x in enumerate(self.lists[self.key]) if x is not None}
        
    def assignOperations(self, name, *, load=I, save=I, keep=I, keepSaved=None, keepClient=None):
        if keepClient is None:
            keepClient = keep
            
        #NOTE #TODE? the method names below conflict with the local vars up here, and current Python syntax takes
        # the variables up here. Should this change the code below will stop working. But this is the "nicest" way to do it
        class the(O()):
            def load(*, i, keep=True):
                if not load:
                    return None
                read = self.file.io[name].read(i=i)
                x = load(read) if read is not None else None
                if keep and self.op[name].keep:
                    self.lists[name][i] = self.op[name].keep(x, i=i)
                return x
            
            def save(x, *, i, keep=True):
                if not save:
                    return
                saved_x = save(x)
                self.file.io[name].write(saved_x, i=i)
                if keep and self.op[name].keep:
                    try:
                        self.lists[name][i] = self.op[name].keep(saved_x, i=i, saved=True)
                        assert self.lists[name][i] is not None
                    except (TypeError, AssertionError):
                        self.lists[name][i] = self.op[name].keep(x, i=i, client=True)
                
            def keep(x, *, i, saved=False, client=False):
                if x is None:
                    return None
                assert not (saved and client), "only one of `saved` and `client` can be specified"
                if not any([keep, keepSaved, keepClient]):
                    return None
                kept = None
                if saved and keepSaved:
                    kept = keepSaved(x)
                elif client and keepClient:
                    kept = keepClient(x)
                elif not saved and not client and keep:
                    kept = keep(x)
                if kept is not None:
                    self.lists[name][i] = kept
                return kept
            
        if not load:
            the.load = False
        if not save:
            the.save = False
        if not any([keep, keepSaved, keepClient]):
            the.keep = False
            
        self.op[name] = the
        if name not in self.lists:
            self.lists[name] = [None] * self.nextIndex
    
    
@staticmethod
def __IndexDataStore__from_specs(specs, **kwargs):
    for k, v in kwargs.items():
        specs[k] = v
    
    f = IndexFileSystem(specs.dir, specs.key)
    d = IndexDataStore(f, _factory_=True)
    
    _readwriteformat = {'read', 'write', 'format'}
    _loadsavekeep = {'load', 'save', 'keep', 'keepSaved', 'keepClient'}
    for name, val in dict.items(specs.op):
        f.assignIO(name, **{a: b for a,b in dict.items(val) if a in _readwriteformat})
        d.assignOperations(name, **{a: b for a,b in dict.items(val) if a in _loadsavekeep})
        
    keyFuncClient = specs.keyFuncClient if 'keyFuncClient' in specs else specs.keyFunc
    keyFunc = lambda keyVal, *, client=False: keyFuncClient(keyVal) if client else specs.keyFunc(keyVal)
    d.assignKey(specs.key, keyFunc)
    
    d.load()
    return d
IndexDataStore.from_specs = __IndexDataStore__from_specs


def json_default(o):
    if isinstance(o, np.int64):
        return int(o)  
    raise TypeError
    
class __IndexDataStore__SpecsHelper(metaclass=staticclass):
    def json(op):
        op.format = 'text'
        op.read = json.loads
        op.write = lambda x: json.dumps(x, default=json_default)
        return op
    def pickle(op):
        op.format = 'bytes'
        op.read = pickle.loads
        op.write = pickle.dumps
        return op
IndexDataStore.SpecsHelper = __IndexDataStore__SpecsHelper
IDSSH = IndexDataStore.SpecsHelper